The purpose of this document is to create a comparison between two operation modes <br>
   - all counters follow the same rule<br>
   - Common Use Self Bag Drop only opens earlier<br>

# import librairies

In [1]:
import datetime
import math
import os
import random
import time

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
from scipy.optimize import OptimizeResult, brute, minimize, minimize_scalar
from tqdm.notebook import tqdm

from src.profiles import generate_dep_Pax_Counters
from src.simfunc.KIX_T1d import (KIX_T1_departure_sim_function,
                                 univariate_cost_function_generator_t1d_N)
from src.simfunc.KIX_T1d_CUSBD import (
    KIX_T1_CUSBD_departure_sim_function,
    univariate_cost_function_generator_t1d_CUSBD_N)
from src.utils import custcallback, custmin

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [3]:
# parameters from "assumption all dept.xlsx" on onedrive
path_onedrive = r"C:\Users\J00638\関西エアポートグループ\Technical HQ department - Masterplan - Masterplan\ADRM model\assumptions all dept\assumptions all dept.xlsx"
df_param_onedrive = pd.read_excel(
    path_onedrive,
    sheet_name="dynamic",
)

# generate df_set_param_T1d_from_drive from one drive excel file (assumptions all dept)
df_param_input = df_param_onedrive.set_index("item", drop=True, inplace=False)

# keep only T1 Int'l dep results
mask_T1d = (df_param_input["terminal"] == "T1") & (df_param_input["Direction"] == "D")
df_param_input_T1 = df_param_input[mask_T1d]

df_param_input_T1 = df_param_input_T1.drop(
    columns=["unit", "terminal", "type", "remarks", "Direction"]
)

# take values from assumption files and put them in a dictionnary
list_item = [
    "target_peak_STD",
    "Pt_checkin_1step_counter",
    "Pt_checkin_2step_counter",
    "N_kiosk",
    "Pt_kiosk",
    "N_security_lanes",
    "Pt_security_lanes",
    "N_emigration_counter",
    "Pt_emigration_counter",
    "N_emigration_self",
    "Pt_emigration_self",
    "modern_pax_ratio",
    "digital_pax_ratio",
    "premium_pax_ratio",
    "start_special_pax_ratio",
    "end_special_pax_ratio",
]

dct_set_param_T1d_from_drive = {
    item: list(df_param_input_T1.loc[item, :]) for item in list_item
}
len_dct_set_param_T1d_from_drive = len(dct_set_param_T1d_from_drive["target_peak_STD"])
dct_set_param_T1d_from_drive["target_peak"] = dct_set_param_T1d_from_drive.pop(
    "target_peak_STD"
)

# define values that are not in assumption files
dct_set_param_T1d_from_drive["freq"] = [
    "1min" for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["win"] = [
    15 for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["show_loading"] = [
    True for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["show_graph"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["save_graph"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["save_xls"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]

# change the start & end special pax, it was a bad idea

dct_set_param_T1d_from_drive["start_special_pax_ratio"] = [
    0.01 for i in range(len_dct_set_param_T1d_from_drive)
]

dct_set_param_T1d_from_drive["end_special_pax_ratio"] = [
    0.99 for i in range(len_dct_set_param_T1d_from_drive)
]

df_set_param_T1d_from_drive = pd.DataFrame.from_dict(dct_set_param_T1d_from_drive)

df_set_param_T1d = df_set_param_T1d_from_drive.copy()
dct_set_param_T1d = dct_set_param_T1d_from_drive.copy()

# create the dictionary of param for the run from the dct_set_param
# we take the values for 2025
dct_run_param = {keys: value[5] for keys, value in dct_set_param_T1d.items()}

<IPython.core.display.Javascript object>

# Calculations for report

## helper function for EBS and LBC capac

In [146]:
def calculate_EBS_LBC(
    df_result,
    MUP_open_time=pd.Timedelta(hours=2, minutes=10),
    MUP_close_time=pd.Timedelta(hours=1),
):
    # change the times after midnight to the next day to calculate properly
    end = pd.Timestamp("2020-10-13 02:00:00")
    mask_late_flight = df_result["STD"] < end
    df_result.loc[mask_late_flight, "STD"] += pd.Timedelta(days=1)

    # mask for bags who will use EBS
    # we take start security queue because some come from checkin and some from CUSBD
    # we should also remove no bag pax (TBD)

    mask_EBS = df_result["start_security_queue"] < df_result["STD"] - MUP_open_time
    df_result.loc[mask_EBS, "EBS_in"] = df_result.loc[mask_EBS, "start_security_queue"]
    df_result.loc[mask_EBS, "EBS_out"] = df_result.loc[mask_EBS, "STD"] - MUP_open_time

    plt_in_EBS = (
        df_result.loc[mask_EBS, ["EBS_in", "Pax_N"]]
        .set_index("EBS_in", drop=False)["Pax_N"]
        .resample("15min")
        .agg(["sum"])
        .cumsum()
    )

    plt_out_EBS = (
        df_result.loc[mask_EBS, ["EBS_out", "Pax_N"]]
        .set_index("EBS_out", drop=False)["Pax_N"]
        .resample("15min")
        .agg(["sum"])
        .cumsum()
    )

    EBS_req = (plt_in_EBS - plt_out_EBS).max()

    # mask for bags who will use LBC

    mask_LBC = df_result["start_security_queue"] > df_result["STD"] - MUP_close_time
    df_result.loc[mask_LBC, "LBC_in"] = df_result.loc[mask_LBC, "start_security_queue"]
    df_result.loc[mask_LBC, "LBC_out"] = df_result.loc[mask_LBC, "STD"]

    plt_in_LBC = (
        df_result.loc[mask_LBC, ["LBC_in", "Pax_N"]]
        .set_index("LBC_in", drop=False)["Pax_N"]
        .resample("15min")
        .agg(["sum"])
        .cumsum()
    )

    plt_out_LBC = (
        df_result.loc[mask_LBC, ["LBC_out", "Pax_N"]]
        .set_index("LBC_out", drop=False)["Pax_N"]
        .resample("15min")
        .agg(["sum"])
        .cumsum()
    )

    LBC_req = (plt_in_LBC - plt_out_LBC).max()

    return EBS_req, LBC_req

<IPython.core.display.Javascript object>

In [147]:
def calculate_avg_dwell_time(df_result, offset=pd.Timedelta(minutes=15)):
    """
    we could use: df_result[["end_emigration_self_process", "end_emigration_counter_process"]]
    but as we do not consider immigration here (check-in study)
    let's consider Pax take about 15 minutes to clear immigration
    """
    # change the times after midnight to the next day to calculate properly
    end = pd.Timestamp("2020-10-13 02:00:00")
    mask_late_flight = df_result["STD"] < end
    df_result.loc[mask_late_flight, "STD"] += pd.Timedelta(days=1)

    df_dwell = df_result["STD"] - (df_result["end_security_process"] + offset)

    q_high = df_dwell.quantile(q=0.90)
    q_low = df_dwell.quantile(q=0.10)

    mask_q = (q_low < df_dwell) & (df_dwell < q_high)

    mean = df_dwell[mask_q].mean() / pd.Timedelta(minutes=1)
    top90 = df_dwell.quantile(q=0.9) / pd.Timedelta(minutes=1)

    return df_dwell, mean, top90

<IPython.core.display.Javascript object>

## all counters have new rule

### import dct_param_onderive

In [148]:
# parameters from "assumption all dept.xlsx" on onedrive
path_onedrive = r"C:\Users\J00638\関西エアポートグループ\Technical HQ department - Masterplan - Masterplan\ADRM model\assumptions all dept\assumptions all dept.xlsx"
df_param_onedrive = pd.read_excel(
    path_onedrive,
    sheet_name="dynamic",
)

# generate df_set_param_T1d_from_drive from one drive excel file (assumptions all dept)
df_param_input = df_param_onedrive.set_index("item", drop=True, inplace=False)

# keep only T1 Int'l dep results
mask_T1d = (df_param_input["terminal"] == "T1") & (df_param_input["Direction"] == "D")
df_param_input_T1 = df_param_input[mask_T1d]

df_param_input_T1 = df_param_input_T1.drop(
    columns=["unit", "terminal", "type", "remarks", "Direction"]
)

# take values from assumption files and put them in a dictionnary
list_item = [
    "target_peak_STD",
    "Pt_checkin_1step_counter",
    "Pt_checkin_2step_counter",
    "N_kiosk",
    "Pt_kiosk",
    "N_security_lanes",
    "Pt_security_lanes",
    "N_emigration_counter",
    "Pt_emigration_counter",
    "N_emigration_self",
    "Pt_emigration_self",
    "modern_pax_ratio",
    "digital_pax_ratio",
    "premium_pax_ratio",
    "start_special_pax_ratio",
    "end_special_pax_ratio",
]

dct_set_param_T1d_from_drive = {
    item: list(df_param_input_T1.loc[item, :]) for item in list_item
}
len_dct_set_param_T1d_from_drive = len(dct_set_param_T1d_from_drive["target_peak_STD"])
dct_set_param_T1d_from_drive["target_peak"] = dct_set_param_T1d_from_drive.pop(
    "target_peak_STD"
)

# define values that are not in assumption files
dct_set_param_T1d_from_drive["freq"] = [
    "1min" for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["win"] = [
    15 for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["show_loading"] = [
    True for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["show_graph"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["save_graph"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["save_xls"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]

# change the start & end special pax, it was a bad idea

dct_set_param_T1d_from_drive["start_special_pax_ratio"] = [
    0.01 for i in range(len_dct_set_param_T1d_from_drive)
]

dct_set_param_T1d_from_drive["end_special_pax_ratio"] = [
    0.99 for i in range(len_dct_set_param_T1d_from_drive)
]

df_set_param_T1d_from_drive = pd.DataFrame.from_dict(dct_set_param_T1d_from_drive)

df_set_param_T1d = df_set_param_T1d_from_drive.copy()
dct_set_param_T1d = dct_set_param_T1d_from_drive.copy()

# create the dictionary of param for the run from the dct_set_param
# we take the values for 2025
dct_run_param = {keys: value[5] for keys, value in dct_set_param_T1d.items()}

<IPython.core.display.Javascript object>

### calculate

In [39]:
# define the optimization parameters
dct_target_wait_time = {
    "kiosk": 1.5,  # IATA optimum is 1-2 minutes
    "CUSBD": 3,  # IATA optimum is 1-5 minutes
}

opening_hour_list = [2.5, 3, 3.5, 4]  # we also consider current case
two_step_ratio_list = [0.2, 0.5, 0.8]

system_list = ["N_kiosk"]
all_system_list = [
    "N_kiosk",
    "N_security_lanes",
    "N_emigration_counter",
    "N_emigration_self",
]
other_result_list = ["N_Counters", "waiting_time_counters", "EBS", "LBC"]

# initialize a path
path_dump = "C:\\Users\\J00638\\KAP python\\Airport sim\\set_runs_departureCUSBD\\set_test\\run_checkin_rule_newloliloul"

# initialize df_results
# column_opening_hour = [a for a in opening_hour_list for b in two_step_ratio_list]
# column_two_step_ratio = [a for b in opening_hour_list for a in two_step_ratio_list]

# index_result_opti = range(len(column_opening_hour))
# columns_list = ["opening_hour", "two_step_ratio"] + system_list + other_result_list
# df_result_opti = pd.DataFrame(columns=columns_list, index=index_result_opti)

# df_result_opti["opening_hour"] = column_opening_hour
# df_result_opti["two_step_ratio"] = column_two_step_ratio

# create dct_param_opti
dct_param_opti = dct_run_param.copy()
dct_param_opti.pop("target_peak", None)

# define dct of edited check-in rule
dct_kwargs_rules = {
    2.5: {
        "start_time": 2.5,
        "onecounter_time": 0.75,
        "base_n_counter": 4,
        "seats_per_add_counter": 60,
    },
    3: {
        "start_time": 3,
        "onecounter_time": 0.75,
        "base_n_counter": 3,
        "seats_per_add_counter": 75,
    },
    3.5: {
        "start_time": 3.5,
        "onecounter_time": 0.75,
        "base_n_counter": 3,
        "seats_per_add_counter": 90,
    },
    4: {
        "start_time": 4,
        "onecounter_time": 0.75,
        "base_n_counter": 3,
        "seats_per_add_counter": 105,
    },
}

<IPython.core.display.Javascript object>

In [40]:
for opening_hour in opening_hour_list:
    # generate df_Counters
    
    # retrieve relevant kwargs_rule
    kwargs_rule = dct_kwargs_rules[opening_hour]

    _, df_Counters = generate_dep_Pax_Counters(
        target_peak=3580,
        terminal="T1",
        custom_counter_rule=True,
        **kwargs_rule,
    )

    # store result
    mask = df_result_opti["opening_hour"] == opening_hour
    df_result_opti.loc[mask, "N_Counters"] = df_Counters["total"].max()

    # as we generated base case df_Pax and df_Counters already
    dct_param_opti["df_Pax"] = df_Pax
    dct_param_opti["df_Counters"] = df_Counters

    for two_step_ratio in two_step_ratio_list:
        dct_param_opti["modern_pax_ratio"] = (
            two_step_ratio - 0.05
        )  # digital pax do not change

        for variable in system_list:
            # edit dct_param_opti
            dct_param_opti["path"] = path_dump

            # optimize current system
            # refresh the kwargs (with eventually results from last optimization)
            kwargs = {
                "dct_param_T1d": dct_param_opti,
                "variable_string": variable,  # eg. N_Z, N_kiosk, N_CUSBD
                "target_wait_time": dct_target_wait_time[
                    variable.split("_")[1]
                ],  # target waiting time in minutes for each system, same for all system for now
                "call_n_iter": None,
                "totalpbar": None,
            }

            # options for custom optimizer, first guess is taken from dct_year base value
            options = {
                "guess": int(49),
                "maxfev": 30,
                "bigstep": 10,
                "smallstep": 1,
                "callback": custcallback,
                "tol": 1,
            }

            # run the optimization for that variable
            print(
                "optimization for {} with:\n rule: opening {}hr before\n two_step_ratio = {}".format(
                    variable, opening_hour, two_step_ratio
                )
            )
            f = univariate_cost_function_generator_t1d_N(**kwargs)
            res = custmin(f, **options)
            print(res)

            # store in this loop's dct for next system
            dct_param_opti[variable] = res["x"]

            # store result
            mask = (df_result_opti["opening_hour"] == opening_hour) & (
                df_result_opti["two_step_ratio"] == two_step_ratio
            )
            df_result_opti.loc[mask, variable] = res["x"]

        # run the optimized simulation once to get the waiting time, EBS and LBC numbers
        (
            df_result,
            list_KPI_run,
            dct_hist_wait_time,
            dct_hist_queue_length,
        ) = KIX_T1_departure_sim_function(**dct_param_opti)

        # store result for waiting time
        mask = (df_result_opti["opening_hour"] == opening_hour) & (
            df_result_opti["two_step_ratio"] == two_step_ratio
        )
        df_result_opti.loc[mask, "waiting_time_counters"] = dct_hist_wait_time[
            "checkin_counter"
        ].quantile(q=0.9)

        # store result for EBS and LBC
        EBS_req, LBC_req = calculate_EBS_LBC(df_result)
        df_result_opti.loc[mask, "EBS"] = int(EBS_req)
        df_result_opti.loc[mask, "LBC"] = int(LBC_req)
        
        # store result for dwell time        
        df_dwell, mean, top90 = calculate_avg_dwell_time(df_result)        
        df_result_opti.loc[mask, "dwell_time"] = mean
        df_result_opti.loc[mask, "df_dwell_time"] = df_dwell

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

optimization for N_kiosk with:
 rule: opening 2.5hr before
 two_step_ratio = 0.8


Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

iteration #0:   x=49   error=0.001111111111111118  function evaluated 1 times step taken: 10
     fun: 0.001111111111111118
    nfev: 1
     nit: 0
 success: False
       x: 49


Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

...

optimization for N_kiosk with:
 rule: opening 4hr before
 two_step_ratio = 0.8


Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

iteration #0:   x=49   error=0.001111111111111118  function evaluated 1 times step taken: 10
     fun: 0.001111111111111118
    nfev: 1
     nit: 0
 success: False
       x: 49


Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [145]:
df_result_opti

,opening_hour,two_step_ratio,N_kiosk,N_Counters,waiting_time_counters,EBS,LBC
0,2.5,0.2,13,131,49.5167,1120,0
1,2.5,0.5,31,131,45.4667,1174,0
2,2.5,0.8,49,131,41.8317,1232,0
3,3.0,0.2,13,126,47.2167,1435,0
4,3.0,0.5,31,126,42.6667,1485,0
5,3.0,0.8,49,126,38.6483,1533,0
6,3.5,0.2,13,121,34.2667,2051,11
7,3.5,0.5,31,121,29.7817,2111,0
8,3.5,0.8,49,121,25.7167,2178,0
9,4.0,0.2,13,125,15.2983,2734,0


<IPython.core.display.Javascript object>

### re-calculate EBS, LBS and dwell time

In [192]:
# re run the EBS, LBC and dwell time
opening_hour_list = [2.5, 3, 3.5, 4]  # we also consider current case
two_step_ratio_list = [0.2, 0.5, 0.8]

for opening_hour in opening_hour_list:
    # generate df_Counters

    # retrieve relevant kwargs_rule
    kwargs_rule = dct_kwargs_rules[opening_hour]

    df_Pax, df_Counters = generate_dep_Pax_Counters(
        target_peak=3580,
        terminal="T1",
        custom_counter_rule=True,
        **kwargs_rule,
    )

    # as we generated base case df_Pax and df_Counters already
    dct_param_opti["df_Pax"] = df_Pax
    dct_param_opti["df_Counters"] = df_Counters

    for two_step_ratio in two_step_ratio_list:
        dct_param_opti["modern_pax_ratio"] = (
            two_step_ratio - 0.05
        )  # digital pax do not change

        mask = (df_result_opti["opening_hour"] == opening_hour) & (
            df_result_opti["two_step_ratio"] == two_step_ratio
        )
        dct_param_opti["N_kiosk"] = df_result_opti.loc[mask, "N_kiosk"].values[0]

        (
            df_result,
            list_KPI_run,
            dct_hist_wait_time,
            dct_hist_queue_length,
        ) = KIX_T1_departure_sim_function(**dct_param_opti)

        # store result for EBS and LBC
        EBS_req, LBC_req = calculate_EBS_LBC(df_result)
        df_result_opti.loc[mask, "EBS"] = int(EBS_req)
        df_result_opti.loc[mask, "LBC"] = int(LBC_req)

        # store result for dwell time
        df_dwell, mean, top90 = calculate_avg_dwell_time(df_result)
        df_result_opti.loc[mask, "dwell_time"] = mean
        df_result_opti.loc[mask, "df_dwell_time"] = df_dwell

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

...

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [211]:
df_result_opti

,opening_hour,two_step_ratio,N_kiosk,N_Counters,waiting_time_counters,EBS,LBC,dwell_time,df_dwell_time
0,2.5,0.2,13,131,49.5167,1159,214,111.317240,0 days 02:09:58
1,2.5,0.5,31,131,45.4667,1213,159,113.306147,0 days 01:30:26
2,2.5,0.8,49,131,41.8317,1271,111,115.243283,0 days 01:51:36
3,3.0,0.2,13,126,47.2167,1474,151,126.838795,0 days 02:07:06
4,3.0,0.5,31,126,42.6667,1524,105,128.940204,0 days 02:42:02
5,3.0,0.8,49,126,38.6483,1572,88,130.916207,0 days 02:17:36
6,3.5,0.2,13,121,34.2667,2090,111,144.050033,0 days 02:45:43
7,3.5,0.5,31,121,29.7817,2150,88,145.623610,0 days 03:00:45
8,3.5,0.8,49,121,25.7167,2217,81,147.090782,0 days 02:56:10
9,4.0,0.2,13,125,15.2983,2773,81,157.101261,0 days 03:39:57


<IPython.core.display.Javascript object>

In [ ]:
df_result_opti_all_counters = df_result_opti[
    [
        "opening_hour",
        "two_step_ratio",
        "N_kiosk",
        "N_Counters",
        "waiting_time_counters",
        "EBS",
        "LBC",
        "dwell_time",
    ]
].copy()

In [217]:
df_result_opti_all_counters

,opening_hour,two_step_ratio,N_kiosk,N_Counters,waiting_time_counters,EBS,LBC,dwell_time
0,2.5,0.2,13,131,49.5167,1159,214,111.317240
1,2.5,0.5,31,131,45.4667,1213,159,113.306147
2,2.5,0.8,49,131,41.8317,1271,111,115.243283
3,3.0,0.2,13,126,47.2167,1474,151,126.838795
4,3.0,0.5,31,126,42.6667,1524,105,128.940204
5,3.0,0.8,49,126,38.6483,1572,88,130.916207
6,3.5,0.2,13,121,34.2667,2090,111,144.050033
7,3.5,0.5,31,121,29.7817,2150,88,145.623610
8,3.5,0.8,49,121,25.7167,2217,81,147.090782
9,4.0,0.2,13,125,15.2983,2773,81,157.101261


<IPython.core.display.Javascript object>

## CUSBD

### import dct_param_onderive

In [11]:
# parameters from "assumption all dept.xlsx" on onedrive
path_onedrive = r"C:\Users\J00638\関西エアポートグループ\Technical HQ department - Masterplan - Masterplan\ADRM model\assumptions all dept\assumptions all dept.xlsx"
df_param_onedrive = pd.read_excel(
    path_onedrive,
    sheet_name="dynamic",
)

# generate df_set_param_T1d_from_drive from one drive excel file (assumptions all dept)
df_param_input = df_param_onedrive.set_index("item", drop=True, inplace=False)

# keep only T1 Int'l dep results
mask_T1d = (df_param_input["terminal"] == "T1") & (df_param_input["Direction"] == "D")
df_param_input_T1 = df_param_input[mask_T1d]

df_param_input_T1 = df_param_input_T1.drop(
    columns=["unit", "terminal", "type", "remarks", "Direction"]
)

# take values from assumption files and put them in a dictionnary
list_item = [
    "target_peak_STD",
    "Pt_checkin_1step_counter",
    "Pt_checkin_2step_counter",
    "N_kiosk",
    "Pt_kiosk",
    "N_security_lanes",
    "Pt_security_lanes",
    "N_emigration_counter",
    "Pt_emigration_counter",
    "N_emigration_self",
    "Pt_emigration_self",
    "modern_pax_ratio",
    "digital_pax_ratio",
    "premium_pax_ratio",
    "start_special_pax_ratio",
    "end_special_pax_ratio",
]

dct_set_param_T1d_from_drive = {
    item: list(df_param_input_T1.loc[item, :]) for item in list_item
}
len_dct_set_param_T1d_from_drive = len(dct_set_param_T1d_from_drive["target_peak_STD"])
dct_set_param_T1d_from_drive["target_peak"] = dct_set_param_T1d_from_drive.pop(
    "target_peak_STD"
)

# define values that are not in assumption files
dct_set_param_T1d_from_drive["freq"] = [
    "1min" for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["win"] = [
    15 for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["show_loading"] = [
    True for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["show_graph"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["save_graph"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]
dct_set_param_T1d_from_drive["save_xls"] = [
    False for i in range(len_dct_set_param_T1d_from_drive)
]

# change the start & end special pax, it was a bad idea

dct_set_param_T1d_from_drive["start_special_pax_ratio"] = [
    0.01 for i in range(len_dct_set_param_T1d_from_drive)
]

dct_set_param_T1d_from_drive["end_special_pax_ratio"] = [
    0.99 for i in range(len_dct_set_param_T1d_from_drive)
]

df_set_param_T1d_from_drive = pd.DataFrame.from_dict(dct_set_param_T1d_from_drive)

df_set_param_T1d = df_set_param_T1d_from_drive.copy()
dct_set_param_T1d = dct_set_param_T1d_from_drive.copy()

# create the dictionary of param for the run from the dct_set_param
# we take the values for 2025
dct_run_param = {keys: value[5] for keys, value in dct_set_param_T1d.items()}

<IPython.core.display.Javascript object>

### calculate

In [42]:
# define the optimization parameters
dct_target_wait_time = {
    "kiosk": 1.5,  # IATA optimum is 1-2 minutes
    "CUSBD": 3,  # IATA optimum is 1-5 minutes
}

opening_hour_list = [3, 3.5, 4, 12]  # we also consider current case and 12 hours before
two_step_ratio_list = [0.8]#[0.2, 0.5, 0.8]

system_list = ["N_kiosk","N_CUSBD"]
all_system_list = [
    "N_kiosk",
    "N_security_lanes",
    "N_emigration_counter",
    "N_emigration_self",
]
other_result_list = ["N_Counters", "waiting_time_counters", "EBS", "LBC"]

# initialize a path
path_dump = "C:\\Users\\J00638\\KAP python\\Airport sim\\set_runs_departureCUSBD\\set_test\\run_checkin_rule_newloliloul"

# initialize df_results
column_opening_hour = [a for a in opening_hour_list for b in two_step_ratio_list]
column_two_step_ratio = [a for b in opening_hour_list for a in two_step_ratio_list]

index_result_opti = range(len(column_opening_hour))
columns_list = ["opening_hour", "two_step_ratio"] + system_list + other_result_list
df_result_opti_CUSBD = pd.DataFrame(columns=columns_list, index=index_result_opti)

df_result_opti_CUSBD["opening_hour"] = column_opening_hour
df_result_opti_CUSBD["two_step_ratio"] = column_two_step_ratio

# create dct_param_opti_CUSBD
dct_param_opti_CUSBD = dct_run_param.copy()
dct_param_opti_CUSBD.pop("target_peak", None)
dct_param_opti_CUSBD['N_CUSBD'] = 24
dct_param_opti_CUSBD['CUSBD_opening_duration'] = 3*60


<IPython.core.display.Javascript object>

In [ ]:
for opening_hour in opening_hour_list:

    dct_param_opti_CUSBD["CUSBD_opening_duration"] = opening_hour * 60

    for two_step_ratio in two_step_ratio_list:
        dct_param_opti_CUSBD["modern_pax_ratio"] = (
            two_step_ratio - 0.05
        )  # digital pax do not change

        # retrieve relevant kwargs_rule
        kwargs_rule = {
            "start_time": 2.5,
            "onecounter_time": 0.75,
            "base_n_counter": math.ceil(3 * (1 - two_step_ratio)),
            "seats_per_add_counter": math.ceil(60 / (1 - two_step_ratio)),
        }

        df_Pax, df_Counters = generate_dep_Pax_Counters(
            target_peak=3580,
            terminal="T1",
            custom_counter_rule=True,
            **kwargs_rule,
        )

        # store result
        mask = (df_result_opti_CUSBD["opening_hour"] == opening_hour) & (
            df_result_opti_CUSBD["two_step_ratio"] == two_step_ratio
        )
        df_result_opti_CUSBD.loc[mask, "N_Counters"] = df_Counters["total"].max()

        # as we generated base case df_Pax and df_Counters already
        dct_param_opti_CUSBD["df_Pax"] = df_Pax
        dct_param_opti_CUSBD["df_Counters"] = df_Counters

        for variable in system_list:
            # edit dct_param_opti_CUSBD
            dct_param_opti_CUSBD["path"] = path_dump

            # optimize current system
            # refresh the kwargs (with eventually results from last optimization)
            kwargs = {
                "dct_param_T1d": dct_param_opti_CUSBD,
                "variable_string": variable,  # eg. N_Z, N_kiosk, N_CUSBD
                "target_wait_time": dct_target_wait_time[
                    variable.split("_")[1]
                ],  # target waiting time in minutes for each system, same for all system for now
                "call_n_iter": None,
                "totalpbar": None,
            }

            # options for custom optimizer, first guess is taken from dct_year base value
            options = {
                "guess": int(49),
                "maxfev": 30,
                "bigstep": 10,
                "smallstep": 1,
                "callback": custcallback,
                "tol": 1,
            }

            # run the optimization for that variable
            print(
                "optimization for {} with:\n rule: opening {}hr before\n two_step_ratio = {}".format(
                    variable, opening_hour, two_step_ratio
                )
            )
            f = univariate_cost_function_generator_t1d_CUSBD_N(**kwargs)
            res = custmin(f, **options)
            print(res)

            # store in this loop's dct for next system
            dct_param_opti_CUSBD[variable] = res["x"]

            # store result
            mask = (df_result_opti_CUSBD["opening_hour"] == opening_hour) & (
                df_result_opti_CUSBD["two_step_ratio"] == two_step_ratio
            )
            df_result_opti_CUSBD.loc[mask, variable] = res["x"]

        # run the optimized simulation once to get the waiting time, EBS and LBC numbers
        (
            df_result,
            list_KPI_run,
            dct_hist_wait_time,
            dct_hist_queue_length,
        ) = KIX_T1_CUSBD_departure_sim_function(**dct_param_opti_CUSBD)

        # store result for waiting time
        mask = (df_result_opti_CUSBD["opening_hour"] == opening_hour) & (
            df_result_opti_CUSBD["two_step_ratio"] == two_step_ratio
        )
        df_result_opti_CUSBD.loc[mask, "waiting_time_counters"] = dct_hist_wait_time[
            "checkin_counter"
        ].quantile(q=0.9)

        # store result for EBS and LBC
        EBS_req, LBC_req = calculate_EBS_LBC(df_result)
        df_result_opti_CUSBD.loc[mask, "EBS"] = int(EBS_req)
        df_result_opti_CUSBD.loc[mask, "LBC"] = int(LBC_req)

        # store result for dwell time
        df_dwell, mean, top90 = calculate_avg_dwell_time(df_result)
        df_result_opti_CUSBD.loc[mask, "dwell_time"] = mean
        df_result_opti_CUSBD.loc[mask, "df_dwell_time"] = df_dwell

In [44]:
df_result_opti_CUSBD_run3 = df_result_opti_CUSBD.copy()
df_result_opti_CUSBD_run3

,opening_hour,two_step_ratio,N_kiosk,N_CUSBD,N_Counters,waiting_time_counters,EBS,LBC
0,3.0,0.8,49,49,33,62.01,1610,0
1,3.5,0.8,49,46,33,62.01,2082,0
2,4.0,0.8,49,42,33,62.01,2360,0
3,12.0,0.8,49,40,33,62.01,2554,0


<IPython.core.display.Javascript object>

In [ ]:
df_result_opti_CUSBD_run4

In [154]:
path_test = r"C:\Users\J00638\Desktop\opti_results.xlsx"

data_test = pd.read_excel(path_test, sheet_name="Sheet2")
data_test

,opening_hour,two_step_ratio,N_kiosk,N_CUSBD,N_Counters,waiting_time_counters,EBS,LBC
0,2.5,0.2,13,15,101,55.0833,937,0
1,2.5,0.5,31,35,68,58.6000,915,0
2,2.5,0.8,49,55,33,62.0100,946,0
3,3.0,0.2,13,13,101,55.0833,1143,0
4,3.0,0.5,31,33,68,58.6000,1325,0
5,3.0,0.8,49,49,33,62.0100,1610,0
6,3.5,0.2,13,12,101,55.0833,1252,0
7,3.5,0.5,31,29,68,58.6000,1597,0
8,3.5,0.8,49,46,33,62.0100,2082,0
9,4.0,0.2,13,11,101,55.0833,1272,0


<IPython.core.display.Javascript object>

### re-calculate EBS, LBC and dwell time

In [194]:
df_result_opti_CUSB = data_test

<IPython.core.display.Javascript object>

In [204]:
for opening_hour in opening_hour_list:

    dct_param_opti_CUSBD["CUSBD_opening_duration"] = opening_hour * 60

    for two_step_ratio in two_step_ratio_list:
        dct_param_opti_CUSBD["modern_pax_ratio"] = (
            two_step_ratio - 0.05
        )  # digital pax do not change

        # input N_kiosk and N_CUSBD from df_result
        mask = (df_result_opti_CUSBD["opening_hour"] == opening_hour) & (
            df_result_opti_CUSBD["two_step_ratio"] == two_step_ratio
        )
        dct_param_opti_CUSBD["N_kiosk"] = df_result_opti_CUSBD.loc[
            mask, "N_kiosk"
        ].values[0]

        dct_param_opti_CUSBD["N_CUSBD"] = df_result_opti_CUSBD.loc[
            mask, "N_CUSBD"
        ].values[0]

        # retrieve relevant kwargs_rule
        kwargs_rule = {
            "start_time": 2.5,
            "onecounter_time": 0.75,
            "base_n_counter": math.ceil(3 * (1 - two_step_ratio)),
            "seats_per_add_counter": math.ceil(60 / (1 - two_step_ratio)),
        }

        df_Pax, df_Counters = generate_dep_Pax_Counters(
            target_peak=3580,
            terminal="T1",
            custom_counter_rule=True,
            **kwargs_rule,
        )

        # store result
        mask = (df_result_opti_CUSBD["opening_hour"] == opening_hour) & (
            df_result_opti_CUSBD["two_step_ratio"] == two_step_ratio
        )

        # as we generated base case df_Pax and df_Counters already
        dct_param_opti_CUSBD["df_Pax"] = df_Pax
        dct_param_opti_CUSBD["df_Counters"] = df_Counters

        # run the optimized simulation once to get the waiting time, EBS and LBC numbers
        (
            df_result,
            list_KPI_run,
            dct_hist_wait_time,
            dct_hist_queue_length,
        ) = KIX_T1_CUSBD_departure_sim_function(**dct_param_opti_CUSBD)

        # store result for waiting time
        mask = (df_result_opti_CUSBD["opening_hour"] == opening_hour) & (
            df_result_opti_CUSBD["two_step_ratio"] == two_step_ratio
        )
        df_result_opti_CUSBD.loc[mask, "waiting_time_counters"] = dct_hist_wait_time[
            "checkin_counter"
        ].quantile(q=0.9)

        # store result for EBS and LBC
        EBS_req, LBC_req = calculate_EBS_LBC(df_result)
        df_result_opti_CUSBD.loc[mask, "EBS"] = int(EBS_req)
        df_result_opti_CUSBD.loc[mask, "LBC"] = int(LBC_req)

        # store result for dwell time
        df_dwell, mean, top90 = calculate_avg_dwell_time(df_result)
        df_result_opti_CUSBD.loc[mask, "dwell_time"] = mean
        df_result_opti_CUSBD.loc[mask, "df_dwell_time"] = df_dwell

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

...

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

Pax and counter generation...:   0%|          | 0/2 [00:00<?, ?it/s]

Simulation running...:   0%|          | 0/1440 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [207]:
df_result_opti_CUSBD.groupby(["opening_hour", "two_step_ratio"]).sum()

N_kiosk  N_CUSBD  N_Counters  \
opening_hour two_step_ratio                                 
2.5          0.2                  13       15         101   
             0.5                  31       35          68   
             0.8                  49       55          33   
3.0          0.2                  13       13         101   
             0.5                  31       33          68   
             0.8                  49       49          33   
3.5          0.2                  13       12         101   
             0.5                  31       29          68   
             0.8                  49       46          33   
4.0          0.2                  13       11         101   
             0.5                  31       27          68   
             0.8                  49       42          33   
12.0         0.2                  13       10         101   
             0.5                  31       25          68   
             0.8                  49       40          33   

                             waiting_time_counters   EBS  LBC  dwell_time  
opening_hour two_step_ratio                                                
2.5          0.2                         55.083333   976  235  110.646846  
             0.5                         58.600000   954  183  115.068665  
             0.8                         62.010000   985  125  120.953924  
3.0          0.2                         55.083333  1182  235  114.931195  
             0.5                         58.600000  1364  183  125.997661  
             0.8                         62.010000  1649  126  136.659662  
3.5          0.2                         55.083333  1291  235  117.640225  
             0.5                         58.600000  1636  183  132.165222  
             0.8                         62.010000  2121  125  146.484353  
4.0          0.2                         55.083333  1311  235  117.976675  
             0.5                         58.600000  1793  183  134.288409  
             0.8                         62.010000  2399  125  150.339122  
12.0         0.2                         55.083333  1344  235  117.884485  
             0.5                         58.600000  1887  183  134.126308  
             0.8                         62.010000  2593  125  150.972565

<IPython.core.display.Javascript object>

In [252]:
mask = df_result_opti_CUSBD["opening_hour"] != 12
df_CUSBD = (
    df_result_opti_CUSBD[mask]
    .fillna("")
    .groupby(["opening_hour", "two_step_ratio"])
    .agg("sum")
)

<IPython.core.display.Javascript object>

In [249]:
df_all_counters = (
    df_result_opti[
        [
            "opening_hour",
            "two_step_ratio",
            "N_kiosk",
            "CUSBD",
            "N_Counters",
            "waiting_time_counters",
            "EBS",
            "LBC",
            "dwell_time",
            "df_dwell_time",
        ]
    ]
    .fillna("")
    .groupby(["opening_hour", "two_step_ratio"])
    .agg("sum")
)

<IPython.core.display.Javascript object>

In [272]:
writer = pd.ExcelWriter(
    r"C:\Users\J00638\KAP python\Airport sim\opti_results.xlsx",
)

<IPython.core.display.Javascript object>

In [273]:
df_CUSBD.to_excel(writer, sheet_name="CUSBD")
df_all_counters.to_excel(writer, sheet_name="all_counters")
writer.save()

<IPython.core.display.Javascript object>

In [255]:
pd.DataFrame(df_all_counters)

N_kiosk  CUSBD  N_Counters  \
opening_hour two_step_ratio                               
2.5          0.2                  13      0       131.0   
             0.5                  31      0       131.0   
             0.8                  49      0       131.0   
3.0          0.2                  13      0       126.0   
             0.5                  31      0       126.0   
             0.8                  49      0       126.0   
3.5          0.2                  13      0       121.0   
             0.5                  31      0       121.0   
             0.8                  49      0       121.0   
4.0          0.2                  13      0       125.0   
             0.5                  31      0       125.0   
             0.8                  49      0       125.0   

                             waiting_time_counters   EBS  LBC  dwell_time  
opening_hour two_step_ratio                                                
2.5          0.2                         49.516667  1159  214  111.317240  
             0.5                         45.466667  1213  159  113.306147  
             0.8                         41.831667  1271  111  115.243283  
3.0          0.2                         47.216667  1474  151  126.838795  
             0.5                         42.666667  1524  105  128.940204  
             0.8                         38.648333  1572   88  130.916207  
3.5          0.2                         34.266667  2090  111  144.050033  
             0.5                         29.781667  2150   88  145.623610  
             0.8                         25.716667  2217   81  147.090782  
4.0          0.2                         15.298333  2773   81  157.101261  
             0.5                         11.865000  2795   82  157.702864  
             0.8                          9.066667  2829   81  158.193951

<IPython.core.display.Javascript object>

In [267]:
df_result_opti_output.to_excel(writer, sheet_name="CUSBD")

<IPython.core.display.Javascript object>

In [263]:
df_result_opti_CUSB

,opening_hour,two_step_ratio,N_kiosk,N_CUSBD,N_Counters,waiting_time_counters,EBS,LBC,dwell_time,df_dwell_time
0,2.5,0.2,13,15,101,55.083333,976,235,110.646846,0 days 02:06:45
1,2.5,0.5,31,35,68,58.600000,954,183,115.068665,0 days 01:19:24
2,2.5,0.8,49,55,33,62.010000,985,125,120.953924,0 days 02:08:43
3,3.0,0.2,13,13,101,55.083333,1182,235,114.931195,0 days 02:36:57
4,3.0,0.5,31,33,68,58.600000,1364,183,125.997661,0 days 02:09:54
5,3.0,0.8,49,49,33,62.010000,1649,126,136.659662,0 days 02:38:19
6,3.5,0.2,13,12,101,55.083333,1291,235,117.640225,0 days 03:08:58
7,3.5,0.5,31,29,68,58.600000,1636,183,132.165222,0 days 01:49:51
8,3.5,0.8,49,46,33,62.010000,2121,125,146.484353,0 days 03:09:38
9,4.0,0.2,13,11,101,55.083333,1311,235,117.976675,0 days 02:04:37


<IPython.core.display.Javascript object>